# Modifying FastPitch to Train on a Non-English (German) Dataset

This notebook is designed to provide a guide on how to train FastPitch on a German dataset from scratch as part of the TTS pipeline. It contains the following sections:
  1. **Introduction**: FastPitch and HiFi-GAN in NeMo
  2. **Dataset Preparation**: How to prepare German dataset for FastPitch
  3. **Training**: Example of FastPitch training and evaluation
  4. **Finetuning HiFi-GAN**: Improving speech quality by Finetuning HiFi-GAN on synthesized mel-spectrograms from FastPitch

# License

> Copyright 2023 NVIDIA. All Rights Reserved.
>
> Licensed under the Apache License, Version 2.0 (the "License");
> you may not use this file except in compliance with the License.
> You may obtain a copy of the License at
>
>     http://www.apache.org/licenses/LICENSE-2.0
>
> Unless required by applicable law or agreed to in writing, software
> distributed under the License is distributed on an "AS IS" BASIS,
> WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
> See the License for the specific language governing permissions and
> limitations under the License.

In [7]:
# """
# You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

# Instructions for setting up Colab are as follows:
# 1. Open a new Python 3 notebook.
# 2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL).
# 3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator).
# 4. Run this cell to set up dependencies.
# 5. Restart the runtime (Runtime -> Restart Runtime) for any upgraded packages to take effect.
# """

# # If you're using Google Colab and not running locally, run this cell.

# ## Install dependencies
# !apt update
# !apt-get install -y sox libsndfile1 ffmpeg
# !pip install wget text-unidecode matplotlib>=3.3.2

# ## Install NeMo
# BRANCH = 'main'
# !python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

# # ## Install pynini
# # !wget https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/nemo_text_processing/install_pynini.sh
# # !bash install_pynini.sh

# !pip install pynini

# """
# Remember to restart the runtime for the kernel to pick up any upgraded packages (e.g. matplotlib)!
# Alternatively, you can uncomment the exit() below to crash and restart the kernel, in the case
# that you want to use the "Run All Cells" (or similar) option.
# """
# # exit()

In [2]:
import json
import nemo
import torch
import librosa
import numpy as np

from pathlib import Path
from tqdm.notebook import tqdm

In [165]:
# # let's download the files we need to run this tutorial
# !rm -rf NeMoGermanTTS
# !mkdir NeMoGermanTTS
# !cd NeMoGermanTTS && wget https://raw.githubusercontent.com/nvidia/NeMo/$BRANCH/scripts/dataset_processing/tts/thorsten_neutral/get_data.py
# !cd NeMoGermanTTS && wget https://raw.githubusercontent.com/nvidia/NeMo/$BRANCH/scripts/dataset_processing/tts/thorsten_neutral/ds_conf/ds_for_fastpitch_align.yaml
# !cd NeMoGermanTTS && wget https://raw.githubusercontent.com/nvidia/NeMo/$BRANCH/examples/tts/fastpitch.py
# !cd NeMoGermanTTS && wget https://raw.githubusercontent.com/nvidia/NeMo/$BRANCH/examples/tts/hifigan_finetune.py
# !cd NeMoGermanTTS && wget https://raw.githubusercontent.com/nvidia/NeMo/$BRANCH/scripts/dataset_processing/tts/extract_sup_data.py
# !cd NeMoGermanTTS && wget https://raw.githubusercontent.com/nvidia/NeMo/$BRANCH/scripts/dataset_processing/tts/generate_mels.py
# !cd NeMoGermanTTS && wget https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/tts/conf/de/fastpitch_align_22050_grapheme.yaml
# !cd NeMoGermanTTS && wget https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/tts/conf/hifigan/hifigan.yaml
# !cd NeMoGermanTTS && mkdir -p model/train_ds && cd model/train_ds && wget https://raw.githubusercontent.com/nvidia/NeMo/$BRANCH/examples/tts/conf/hifigan/model/train_ds/train_ds_finetune.yaml
# !cd NeMoGermanTTS && mkdir -p model/validation_ds && cd model/validation_ds && wget https://raw.githubusercontent.com/nvidia/NeMo/$BRANCH/examples/tts/conf/hifigan/model/validation_ds/val_ds_finetune.yaml
# !cd NeMoGermanTTS && mkdir -p model/generator && cd model/generator && wget https://raw.githubusercontent.com/nvidia/NeMo/$BRANCH/examples/tts/conf/hifigan/model/generator/v1.yaml

# Introduction

### FastPitch

FastPitch is non-autoregressive model for mel-spectrogram generation based on FastSpeech, conditioned on fundamental frequency contours. For more details about model, please refer to the original [paper](https://ieeexplore.ieee.org/abstract/document/9413889). Original [FastPitch model](https://ieeexplore.ieee.org/abstract/document/9413889) uses an external Tacotron 2 model trained on LJSpeech-1.1 to extract training alignments and estimate durations of input symbols. This implementation of FastPitch is based on [Deep Learning Examples](https://github.com/NVIDIA/DeepLearningExamples/tree/master/PyTorch/SpeechSynthesis/FastPitch), which uses an alignment mechanism proposed in [RAD-TTS](https://openreview.net/pdf?id=0NQwnnwAORi) and extended in [TTS Aligner](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9747707).

For more information on training a basic FastPitch model, please refer to [FastPitch_MixerTTS_Training.ipynb](https://github.com/NVIDIA/NeMo/blob/main/tutorials/tts/FastPitch_MixerTTS_Training.ipynb) tutorial.

### HiFiGAN
HiFiGAN is a generative adversarial network (GAN) model that generates audio from mel spectrograms. The generator uses transposed convolutions to upsample mel spectrograms to audio. For more details about the model, please refer to the original [paper](https://arxiv.org/abs/2010.05646). NeMo re-implementation of HiFi-GAN can be found [here](https://github.com/NVIDIA/NeMo/blob/main/nemo/collections/tts/models/hifigan.py).

# Dataset Preparation

We will show example of preprocessing and training using OpenSLR's German Neutral TTS dataset ([link](https://www.openslr.org/95)). It is a free single german speaker dataset (> 23 hours) by Thorsten Müller (voice) and Dominik Kreutz (audio optimization) for tts training.

In this section, we will cover:
1. Downloading the dataset
2. Extracting supplementary data

## 1. Download Thorsten's Neutral Dataset and Create Data Manifests

We've created a script, `scripts/dataset_processing/tts/thorsten_neutral/get_data.py`, to download Thorsten's Datasets and generate train/val/test splits as JSON manifests with the following fields for each datapoint:
1. `audio_filepath`: location of the wav file;
2. `duration`: duration of the wav file;
3. `text`: original text;
4. `normalized_text`: normalized text through our text normalization pipeline.

This script supports processing either of Thorsten's Neutral Datasets 21.02 or 22.10. In this tutorial, we only focus on the latest 22.10 version dataset. Please refer [thorsten-muller-s-german-neutral-tts-datasets](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/main/tts/datasets.html#thorsten-muller-s-german-neutral-tts-datasets) for more details about Thorsten's datasets.

You can run the below command to obtain the final manifests, `train_manifest_text_normed.json`, `val_manifest_text_normed.json` and `test_manifest_text_normed.json`. **Note** that this script would take sometime (~2 hours) to download and normalize the entire dataset.

In [3]:
# # Version 22.10
# !cd datasets/NemoSpanishTTS/ && python get_data.py \
#     --data-root ../../datasets \
#     --manifests-root ./ \
#     # --data-version "22_10" \
#     --val-size 100 \
#     --test-size 100 \
#     --seed-for-ds-split 100 \
#     --num-workers -1 \
#     --normalize-text
    
    # Version 22.10
    
# Funciona

import os
os.chdir("/home/irene/notebooks")

!python es_get_data.py \
    --data-root /home/irene/datasets \
    --manifests-root ../datasets/NemoSpanishTTSEsMapa152Finetuning \
    --val-size 3 \
    --test-size 2 \
    --seed-for-ds-split 87 \
    --num-workers -1 \
    --normalize-text


[NeMo W 2023-08-04 11:51:11 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda9SetDeviceEi'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
      warn(
    
Dataset directory found
[NeMo I 2023-08-04 11:51:14 es_get_data:148] Preparing JSON train/val/test splits.
137it [00:07, 18.84it/s]
5it [00:00, 22.22it/s]
3it [00:00, 15.20it/s]
[NeMo I 2023-08-04 11:51:22 es_get_data:198] Preparing JSON train/val/test splits is complete.
[NeMo I 2023-08-04 11:51:22 es_get_data:106] Saving JSON split to ../datasets/NemoSpanishTTSEsMapa152Finetuning/train_manifest.json.
[NeMo I 2023-08-04 11:51:22 es_get_data:106] Saving J

In [167]:
# data_thorsten_2210 directory looks like
!ls /datasets/datasetCastellanoReducido

19demarzo  metadata_dev.csv  metadata_test.csv	metadata_train.csv


In [168]:
! more /datasets/datasetCastellanoReducido/metadata_dev.csv

19demarzo_3267|Juan de Dios, que al volver de su desmayo era completamente extra
ño a la situación que nos encontrábamos, y no parecía tener ojos ni oídos más qu
e para espectáculos y voces de su propia alma,|Juan de Dios, que al volver de su
 desmayo era completamente extraño a la situación que nos encontrábamos, y no pa
recía tener ojos ni oídos más que para espectáculos y voces de su propia alma,
19demarzo_3268|se adelantó hacia Inés con ademán embarazoso, y le dijo: -Pero Ga
briel la habrá enterado a Vd. de todo. ¿La he ofendido a Vd. en algo?|se adelant
ó hacia Inés con ademán embarazoso, y le dijo: -Pero Gabriel la habrá enterado a
 Usted de todo. ¿La he ofendido a Usted en algo?
19demarzo_3269|Bien habrá comprendido Vd... -Este caballero -dijo Inés- está mue
rto de miedo, y no se moverá de aquí. ¿Quiere Vd. esconderse en la cocina?|Bien 
habrá comprendido Usted -Este caballero -dijo Inés- está muerto de miedo, y no s
e moverá de aquí. ¿Quiere Usted esconderse en la cocina?
19dem

In [169]:
# manifests
!ls ./NemoSpanishTTS/*_text_normed.json

./NemoSpanishTTS/test_manifest_text_normed.json
./NemoSpanishTTS/train_manifest_text_normed.json
./NemoSpanishTTS/val_manifest_text_normed.json


In [53]:
from pydub import AudioSegment
file_path = "/home/irene/datasets/datasetAlfonsoGutierrezFinetuning/asignaturaBueno/resumen_asignatura_bueno0.wav"
sound = AudioSegment.from_wav(file_path)
sound = sound.set_channels(1)
sound.export(file_path, format="wav")

<_io.BufferedRandom name='/home/irene/datasets/datasetAlfonsoGutierrezFinetuning/asignaturaBueno/resumen_asignatura_bueno0.wav'>

In [7]:
#Esto soluciona un error de canales: así si que se extrae la información suplementaria: se pasa de estereo a mono.
from pydub import AudioSegment
import os

folder_path = "/home/irene/datasets/datasetEsMapa152Finetuning/esmapa152"

# Iterate over the files in the folder
# for i in range(92):
    # file_name = f"esmapa{i}.wav" # Formato normal datasetBueno
for i in range(153):
    file_name = f"esmapa{i:04d}.wav"  # Formato con varios 0 DatasetEsMapa
    file_path = os.path.join(folder_path, file_name)

    # Check if the file exists before processing
    if os.path.exists(file_path):
        # Load the audio file
        sound = AudioSegment.from_wav(file_path)

        # Set the number of channels to 1
        sound = sound.set_channels(1)

        # Export the modified audio back to the same file path
        sound.export(file_path, format="wav")

        print(f"Converted file: {file_name}")
    else:
        print(f"File not found: {file_name}")



Converted file: esmapa0000.wav
Converted file: esmapa0001.wav
Converted file: esmapa0002.wav
Converted file: esmapa0003.wav
Converted file: esmapa0004.wav
Converted file: esmapa0005.wav
Converted file: esmapa0006.wav
Converted file: esmapa0007.wav
Converted file: esmapa0008.wav
Converted file: esmapa0009.wav
Converted file: esmapa0010.wav
Converted file: esmapa0011.wav
Converted file: esmapa0012.wav
Converted file: esmapa0013.wav
Converted file: esmapa0014.wav
Converted file: esmapa0015.wav
Converted file: esmapa0016.wav
Converted file: esmapa0017.wav
Converted file: esmapa0018.wav
Converted file: esmapa0019.wav
Converted file: esmapa0020.wav
Converted file: esmapa0021.wav
Converted file: esmapa0022.wav
Converted file: esmapa0023.wav
Converted file: esmapa0024.wav
Converted file: esmapa0025.wav
Converted file: esmapa0026.wav
Converted file: esmapa0027.wav
Converted file: esmapa0028.wav
Converted file: esmapa0029.wav
Converted file: esmapa0030.wav
Converted file: esmapa0031.wav
Converte

## 2. Extracting Supplementary Data

As mentioned in the [FastPitch and MixerTTS training tutorial](FastPitch_MixerTTS_Training.ipynb) - To accelerate and stabilize our training, we also need to extract pitch for every audio, estimate pitch statistics (mean, std, min, and max). To do this, all we need to do is iterate over our data one time, via `extract_sup_data.py` script.

**Note**: This is an optional step, if skipped, it will be automatically executed within the first epoch of training FastPitch.

The configuration remains the same as described in `scripts/dataset_processing/tts/thorsten_neutral/ds_conf/ds_for_fastpitch_align.yaml`. Note that there is no need to specify `whitelist_path` config anymore from NeMo Release 1.17.0 because it has been moved to a new dependency repo https://github.com/NVIDIA/NeMo-text-processing and it has been applied implicitly.

Run below command line and you will obtain the pitch normalization parameters estimated on the entire dataset. It will cost around 1.5 hours.

In [8]:
# !cd /home/irene/datasets/NemoSpanishTTS && HYDRA_FULL_ERROR=1 python extract_sup_data.py \
#         --config-path . \
#         --config-name ds_for_fastpitch_align.yaml \
#         manifest_filepath=train_manifest_text_normed.json \
#         sup_data_path=sup_data \
#         ++dataloader_params.num_workers=4 
        
#FUNCIONA: Había que cambiar los canales, para los audios del profe de Segovia
import glob, os
os.chdir("/home/irene/datasets/NemoSpanishTTSEsMapa152Finetuning")

!python extract_sup_data.py \
        --config-path ./ \
        --config-name ds_for_fastpitch_align.yaml \
        manifest_filepath=train_manifest_text_normed.json \
        sup_data_path=sup_data \
        ++dataloader_params.num_workers=4

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda9SetDeviceEi'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
[NeMo W 2023-08-04 12:14:04 optimizers:66] Could not import distributed_fused_adam optimizer from Apex
[NeMo W 2023-08-04 12:14:04 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
    See https://hydra.cc/docs/next/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
      ret = run_job(
    
[NeMo W 2023-08-04 12:14:06 experimental:27] Module <class 'ne

After running the above command line, you will observe a new folder `NeMoGermanTTS/sup_data/pitch` and printouts of pitch statistics below. Specify these values to the FastPitch training configurations. We will be there in the following section.
```bash
PITCH_MEAN=126.73465728759766, PITCH_STD=38.099849700927734
PITCH_MIN=65.4063949584961, PITCH_MAX=2021.7091064453125

```

# Training

Before we train our model, let's define model config. Most of the model config stays the same as defined here: `examples/tts/conf/de/fastpitch_align_22050_grapheme.yaml`, except that the `pitch_mean` and `pitch_std` should be updated with the values estimated by the above `extract_sup_data.py` script. Note that there is no need to specify `whitelist_path` config anymore from NeMo Release 1.17.0 because it has been moved to a new dependency repo https://github.com/NVIDIA/NeMo-text-processing and it has been applied implicitly.

If you are using Weights and Biases, you may need to login first. More details [here](https://docs.wandb.ai/ref/cli/wandb-login).

In [9]:
!wandb login 7f8717dd64209b51a51493f579c375a7ca34fd2f

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Now we are ready for training our model! Let's try to train FastPitch. Copy and Paste the `PITCH_MEAN` and `PITCH_STD` from previous steps to override `pitch_mean` and `pitch_std` configs below.

In [215]:
# !(cd NemoSpanishTTS && CUDA_VISIBLE_DEVICES=0 python fastpitch.py --config-path . --config-name fastpitch_align_22050_grapheme \
#   model.train_ds.dataloader_params.batch_size=32 \
#     model.validation_ds.dataloader_params.batch_size=32 \
#     train_dataset=train_manifest_text_normed.json \
#     validation_datasets=val_manifest_text_normed.json \
#     sup_data_path=sup_data \
#     exp_manager.exp_dir=resultGermanTTS \
#     trainer.max_epochs=1 \
#     trainer.check_val_every_n_epoch=1 \
#     pitch_mean=126.73465728759766 \
#     pitch_std=38.099849700927734 \
#     +exp_manager.create_wandb_logger=true \
#     +exp_manager.wandb_logger_kwargs.name="tutorial" \
#     +exp_manager.wandb_logger_kwargs.project="GermanTTS")

In [18]:
!pwd

/home/irene/notebooks


In [20]:
!(cd /home/irene/datasets/NemoSpanishTTS && CUDA_VISIBLE_DEVICES=0 python fastpitch.py --config-path . --config-name fastpitch_align_44100 \
  model.train_ds.dataloader_params.batch_size=32 \
    model.validation_ds.dataloader_params.batch_size=32 \
    train_dataset=train_manifest_text_normed.json \
    validation_datasets=val_manifest_text_normed.json \
    sup_data_path=sup_data \
    exp_manager.exp_dir=resultSpanishTTS \
    trainer.check_val_every_n_epoch=1 \
    pitch_mean=126.73465728759766 \
    pitch_std=38.099849700927734 \
    +exp_manager.create_wandb_logger=true \
    +exp_manager.wandb_logger_kwargs.name="tutorial" \
    +exp_manager.wandb_logger_kwargs.project="SpanishTTS")

[NeMo W 2023-06-27 15:47:39 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
2023-06-27 15:47:40.726908: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[NeMo W 2023-06-27 15:47:42 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
    See https://hydra.cc/docs/next/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
      ret = run_job(
    
Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[NeMo 

Note:
1. We use `CUDA_VISIBLE_DEVICES=0` to limit training to single GPU.
2. For debugging you may also add the following flags: `HYDRA_FULL_ERROR=1`, `CUDA_LAUNCH_BLOCKING=1`

<b>Note</b>: We've limited the above run to 1 epoch only, so we can validate the implementation within the scope of this tutorial. We recommend around 5000 epochs when training FastPitch from scratch.

## Evaluating FastPitch + pretrained HiFi-GAN

Let's evaluate the quality of the FastPitch model generated so far using a HiFi-GAN model pre-trained on English.

In [6]:
import IPython.display as ipd
from nemo.collections.tts.models import HifiGanModel, FastPitchModel
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt

[NeMo W 2023-06-28 16:17:24 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.


Extension horovod.torch has not been built: /usr/local/lib/python3.10/dist-packages/horovod/torch/mpi_lib_v2.cpython-310-x86_64-linux-gnu.so not found
If this is not expected, reinstall Horovod with HOROVOD_WITH_PYTORCH=1 to debug the build error.
Warning! MPI libs are missing, but python applications are still available.


2023-06-28 16:17:25.537539: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# test = "das werden einmal \u00fcber eine million kleiner buchenpflanzen werden." # text input to the model
# test_id = "2b2b496ccc9b57130f559c4fd827825f" # identifier for the audio corresponding to the test text
# data_path = "data_thorsten_2210/ThorstenVoice-Dataset-22_10/ThorstenVoice-Dataset_2022.10/wavs/" # path to dataset folder with wav files from original dataset
# seed = 1234
test = "Durante nuestra conversación advertí que la multitud aumentaba, apretándose más." # text input to the model
test_id = "19demarzo_0000" # identifier for the audio corresponding to the test text
data_path = "/datasets/datasetCastellanoReducido/19demarzo/" # path to dataset folder with wav files from original dataset
seed = 1234

In [7]:
def evaluate_spec_fastpitch_ckpt(spec_gen_model, v_model, test):
    with torch.no_grad():
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.enabled = True
        torch.backends.cudnn.benchmark = False
        parsed = spec_gen_model.parse(str_input=test, normalize=True)
        spectrogram = spec_gen_model.generate_spectrogram(tokens=parsed)
        print(spectrogram.size())
        audio = v_model.convert_spectrogram_to_audio(spec=spectrogram)

    spectrogram = spectrogram.to('cpu').numpy()[0]
    audio = audio.to('cpu').numpy()[0]
    audio = audio / np.abs(audio).max()
    return audio, spectrogram

In [11]:
# load fastpitch and hifigan models
import glob, os
fastpitch_model_path = sorted(
    glob.glob("/home/irene/datasets/NemoSpanishTTS/resultSpanishTTS/FastPitch/*/checkpoints/FastPitch.nemo"),
    key=os.path.getmtime
)[-1] # path_to_fastpitch_nemo_or_ckpt
hfg_ngc = "tts_en_lj_hifigan_ft_mixerttsx" # NGC pretrained model name: https://catalog.ngc.nvidia.com/orgs/nvidia/teams/nemo/models/tts_en_lj_hifigan

vocoder_model = HifiGanModel.from_pretrained(hfg_ngc, strict=False).eval().cuda()
if ".nemo" in fastpitch_model_path:
    spec_gen_model = FastPitchModel.restore_from(fastpitch_model_path).eval().cuda()
else:
    spec_gen_model = FastPitchModel.load_from_checkpoint(checkpoint_path=fastpitch_model_path).eval().cuda()

IndexError: list index out of range

In [ ]:
%matplotlib inline

audio, spectrogram = evaluate_spec_fastpitch_ckpt(spec_gen_model, vocoder_model, test)

# visualize the spectrogram
if spectrogram is not None:
    imshow(spectrogram, origin="lower")
    plt.show()

# audio
print("original audio")
ipd.display(ipd.Audio(data_path+test_id+'.wav', rate=22050))
print("predicted audio")
ipd.display(ipd.Audio(audio, rate=22050))

You would hear that the above synthesized audio quality is not as good as we expect. It would be improved after continuing to train 5000 epochs. But again,the quality is still not acceptable. A straightforward solution is to finetune the HiFi-GAN model following the tutorial [FastPitch_Finetuning.ipynb](FastPitch_Finetuning.ipynb). Let's try that out next!

# Finetuning HiFi-GAN

Improving speech quality by finetuning HiFi-GAN on synthesized mel-spectrograms from FastPitch.

## Generating synthetic mels

To generate mel-spectrograms from FastPitch, we can use `generate_spectrogram` method defined in `nemo/collections/tts/models/fastpitch.py`. However, the resulting spectrogram may be different from ground truth mel spectrogram, as shown below:

In [ ]:
test_audio_filepath = "data_thorsten_2210/ThorstenVoice-Dataset-22_10/ThorstenVoice-Dataset_2022.10/wavs/43ee28172fe1a9d1eebd77bc09f03e51.wav"
test_audio_text = "daran beteiligten sich einhundertachtzig kommunen und hochschulen."

In [ ]:
from matplotlib.pyplot import imshow
from nemo.collections.tts.models import FastPitchModel
from matplotlib import pyplot as plt
import librosa
import librosa.display
import torch
import soundfile as sf
import numpy as np
from nemo.collections.tts.parts.utils.tts_dataset_utils import BetaBinomialInterpolator

def load_wav(audio_file):
    with sf.SoundFile(audio_file, 'r') as f:
        samples = f.read(dtype='float32')
    return samples.transpose()

def plot_logspec(spec, axis=None):
    librosa.display.specshow(
        librosa.amplitude_to_db(spec, ref=np.max),
        y_axis='linear',
        x_axis="time",
        fmin=0,
        fmax=8000,
        ax=axis
    )

In [ ]:
spec_model = FastPitchModel.restore_from(fastpitch_model_path).eval().cuda()

So we have 2 types of mel spectrograms that we can use for finetuning HiFi-GAN:

### 1. Original mel spectrogram generated from original audio file

In [ ]:
print("loading original melspec")
y, sr = librosa.load(test_audio_filepath)
# change n_fft, win_length, hop_length parameters below based on your specific config file
spectrogram2 = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=1024, win_length=1024, hop_length=256)
spectrogram = spectrogram2[ :80, :]
print("spectrogram shape = ", spectrogram.shape)
plot_logspec(spectrogram)
plt.show()

### 2. Mel spectrogram predicted from FastPitch

In [ ]:
print("loading fastpitch melspec via generate_spectrogram")
with torch.no_grad():
    text = spec_model.parse(test_audio_text, normalize=False)
    spectrogram = spec_model.generate_spectrogram(
      tokens=text,
      speaker=None,
    )
spectrogram = spectrogram.to('cpu').numpy()[0]
plot_logspec(spectrogram)
print("spectrogram shape = ", spectrogram.shape)
plt.show()

**Note**: The above predicted spectrogram has the duration of 241 frames which is not equal to the ground truth 345 frames. In order to finetune HiFi-GAN we need mel spectrogram predicted from FastPitch with ground truth alignment and duration.

### 2.1 Mel spectrogram predicted from FastPitch with groundtruth alignment and duration

In [ ]:
print("loading fastpitch melspec via forward method with groundtruth alignment and duration")
with torch.no_grad():
    device = spec_model.device
    beta_binomial_interpolator = BetaBinomialInterpolator()
    text = spec_model.parse(test_audio_text, normalize=False)
    text_len = torch.tensor(text.shape[-1], dtype=torch.long, device=device).unsqueeze(0)
    audio = load_wav(test_audio_filepath)
    audio = torch.from_numpy(audio).unsqueeze(0).to(device)
    audio_len = torch.tensor(audio.shape[1], dtype=torch.long, device=device).unsqueeze(0)
    spect, spect_len = spec_model.preprocessor(input_signal=audio, length=audio_len)
    attn_prior = torch.from_numpy(
      beta_binomial_interpolator(spect_len.item(), text_len.item())
    ).unsqueeze(0).to(text.device)
    spectrogram = spec_model.forward(
      text=text,
      input_lens=text_len,
      spec=spect,
      mel_lens=spect_len,
      attn_prior=attn_prior,
      speaker=None,
    )[0]
spectrogram = spectrogram.to('cpu').numpy()[0]
print("spectrogram shape = ", spectrogram.shape)
plot_logspec(spectrogram)
plt.show()

In our experience,
- Finetuning with #1 has artifacts from the original audio (noise) that get passed on as input to the vocoder resulting in artifacts in vocoder output in the form of noise.
- <b> On the other hand, #2.1 (i.e. `Mel spectrogram predicted from FastPitch with groundtruth alignment and duration`) gives the best results because it enables HiFi-GAN to learn mel spectrograms generated by FastPitch as well as duration distributions closer to the real world (i.e. ground truth) durations. </b>

From implementation perspective - we follow the same process described in [Finetuning FastPitch for a new speaker](FastPitch_Finetuning.ipynb) - i.e. take the latest checkpoint from FastPitch training and predict spectrograms for each of the input records in `train_manifest_text_normed.json`, `test_manifest_text_normed.json` and `val_manifest_text_normed.json`. NeMo provides an efficient script, [scripts/dataset_processing/tts/generate_mels.py](https://raw.githubusercontent.com/nvidia/NeMo/main/scripts/dataset_processing/tts/generate_mels.py), to generate Mel-spectrograms in the directory `NeMoGermanTTS/mels` and also create new JSON manifests with a suffix `_mel` by adding a new key `"mel_filepath"`. For example, `train_manifest_text_normed.json` corresponds to `train_manifest_text_normed_mel.json` saved in the same directory. You can run the following CLI to obtain the new JSON manifests.

In [1]:
# !cd NeMoGermanTTS && python generate_mels.py \
#     --cpu \
#     --fastpitch-model-ckpt {fastpitch_model_path.split("/", maxsplit=1)[1]} \
#     --input-json-manifests train_manifest_text_normed.json val_manifest_text_normed.json test_manifest_text_normed.json \
#     --output-json-manifest-root ./
    
!cd NeMoGermanTTS && python generate_mels.py \
    --cpu \
    --fastpitch-model-ckpt {fastpitch_model_path.split("/", maxsplit=1)[1]} \
    --input-json-manifests train_manifest_text_normed.json val_manifest_text_normed.json test_manifest_text_normed.json \
    --output-json-manifest-root ./

/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `cd NeMoGermanTTS && python generate_mels.py      --cpu      --fastpitch-model-ckpt {fastpitch_model_path.split("/", maxsplit=1)[1]}      --input-json-manifests train_manifest_text_normed.json val_manifest_text_normed.json test_manifest_text_normed.json      --output-json-manifest-root ./'


Revisiting how we implement #2.1 (i.e. Predicted mel spectrogram predicted from FastPitch with groundtruth alignment and duration):

1. Notice above that we use audio from dataset (`audio` variable) to compute spectrogram length (`spect_len`):
    ```python
    spect, spect_len = spec_model.preprocessor(input_signal=audio, length=audio_len)
    ```
2. and groundtruth alignment (`attn_prior`).
    ```python
    attn_prior = torch.from_numpy(
          beta_binomial_interpolator(spect_len.item(), text_len.item())
        ).unsqueeze(0).to(text.device)
    ```
3. We use both of them to generate synthetic mel spectrogram via `spec_model.forward` method:
    ```python
    spectrogram = spec_model.forward(
          text=text,
          input_lens=text_len,
          spec=spect,
          mel_lens=spect_len,
          attn_prior=attn_prior,
          speaker=speaker,
        )[0]
    ```

Repeat the above script for train and validation datasets as well.

In [ ]:
# Example HiFi-GAN manifest:
!head -n1 NeMoGermanTTS/train_manifest_text_normed_mel.json | jq

## Launch finetuning

We will be re-using the existing HiFi-GAN config and HiFi-GAN pretrained on English.

In [ ]:
!cd NeMoGermanTTS && CUDA_VISIBLE_DEVICES=0 python hifigan_finetune.py --config-path . --config-name hifigan.yaml \
    model.max_steps=10 \
    model.optim.lr=0.00001 \
    ~model.optim.sched \
    train_dataset=train_manifest_text_normed_mel.json \
    validation_datasets=val_manifest_text_normed_mel.json \
    exp_manager.exp_dir=resultGermanTTS \
    +init_from_pretrained_model={hfg_ngc} \
    +trainer.val_check_interval=5 \
    trainer.check_val_every_n_epoch=null \
        
    model/train_ds=train_ds_finetune \
    model/validation_ds=val_ds_finetune \
    exp_manager.create_wandb_logger=true \
    exp_manager.wandb_logger_kwargs.name="tutorial_2" \
    exp_manager.wandb_logger_kwargs.project="GermanTTS"

<b>Note</b>: We've limited the above run to 10 steps only, so we can validate the implementation within the scope of this tutorial. We recommend evaluating around every 50 steps HiFi-GAN until you get desired quality results.

## Evaluating FastPitch and Finetuned HiFi-GAN

Let's evaluate the quality of the FastPitch model generated so far using a HiFi-GAN model finetuned on predicted mels.

In [ ]:
hfg_path = sorted(glob.glob("NeMoGermanTTS/resultGermanTTS/HifiGan/*/checkpoints/HifiGan.nemo"), key=os.path.getmtime)[-1]

if ".nemo" in hfg_path:
    vocoder_model_pt = HifiGanModel.restore_from(hfg_path).eval().cuda()
else:
    vocoder_model_pt = HifiGanModel.load_from_checkpoint(checkpoint_path=hfg_path).eval().cuda()

if ".nemo" in fastpitch_model_path:
    spec_gen_model = FastPitchModel.restore_from(fastpitch_model_path).eval().cuda()
else:
    spec_gen_model = FastPitchModel.load_from_checkpoint(checkpoint_path=fastpitch_model_path).eval().cuda()

In [ ]:
%matplotlib inline

audio, spectrogram = evaluate_spec_fastpitch_ckpt(spec_gen_model, vocoder_model_pt, test)

# visualize the spectrogram
if spectrogram is not None:
    imshow(spectrogram, origin="lower")
    plt.show()

# audio
print("original audio")
ipd.display(ipd.Audio(data_path+test_id+'.wav', rate=22050))
print("predicted audio")
ipd.display(ipd.Audio(audio, rate=22050))

That's it!